**Created by Sanjay Raghuwanshi ** <br/>
[Github](https://github.com/sanjay-raghu) <br/>
[Linkedin](https://www.linkedin.com/in/sanjayiitg/) <br/>
**Sentiment Analysis:** the process of computationally identifying and categorizing opinions expressed in a piece of text, especially in order to determine whether the writer's attitude towards a particular topic, product, etc. is positive, negative, or neutral.

**What's New** I have added how to deal with data imbalance. Almost all classification task have this problem as number of data of every class if different. For current dataset number of data having positive sentiments is very low relative to data with negative sentiment.

**Solving class imbalaned data**:
- upsampling 
- using class weighted loss function

Using LSTM to classify the movie reviews into positive and negative.


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,classification_report
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

Using TensorFlow backend.


Only keeping the necessary columns.

In [2]:
data = pd.read_csv('../input/Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

Data preview

In [3]:
data.head()

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


Next, I am dropping the 'Neutral' sentiments as my goal was to only differentiate positive and negative tweets. After that, I am filtering the tweets so only valid texts and words remain.  Then, I define the number of max features as 2000 and use Tokenizer to vectorize and convert text into Sequences so the Network can deal with it as input.

In [4]:
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
# removing special chars
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
#
data.head()

,text,sentiment
1,rt scottwalker didnt catch the full gopdebate ...,Positive
3,rt robgeorge that carly fiorina is trending h...,Positive
4,rt danscavino gopdebate w realdonaldtrump deli...,Positive
5,rt gregabbott_tx tedcruz on my first day i wil...,Positive
6,rt warriorwoman91 i liked her and was happy wh...,Negative


In [5]:
print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt','')
data.head()

4472
16986


,text,sentiment
1,scottwalker didnt catch the full gopdebate la...,Positive
3,robgeorge that carly fiorina is trending hou...,Positive
4,danscavino gopdebate w realdonaldtrump delive...,Positive
5,gregabbott_tx tedcruz on my first day i will ...,Positive
6,warriorwoman91 i liked her and was happy when...,Negative


In [6]:
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)
X[:2]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
         359,  120,    1,  692,    2,   39,   58,  234,   37,  207,    6,
         172, 1745,   12, 1308, 1394,  733],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          16,  281,  249,    5,  809,  102,  170,   26,  134,    6,    1,
         171,   12,    2,  231,  713,   17]], dtype=int32)

Next, I compose the LSTM Network. Note that **embed_dim**, **lstm_out**, **batch_size**, **droupout_x** variables are hyperparameters, their values are somehow intuitive, can be and must be played with in order to achieve good results. Please also note that I am using softmax as activation function. The reason is that our Network is using categorical crossentropy, and softmax is just the right activation method for that.

In [7]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 28, 128)           256000    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 28, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


Hereby I declare the train and test dataset.

In [8]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(8583, 28) (8583, 2)
(2146, 28) (2146, 2)


Here we train the Network. We should run much more than 7 epoch, but I would have to wait forever for kaggle, so it is 7 for now.

In [9]:
batch_size = 128
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 1)

Epoch 1/15
8583/8583 [==============================] - 7s 837us/step - loss: 0.4851 - acc: 0.7949
Epoch 2/15
8583/8583 [==============================] - 5s 605us/step - loss: 0.3420 - acc: 0.8554
Epoch 3/15
8583/8583 [==============================] - 5s 604us/step - loss: 0.2960 - acc: 0.8743
Epoch 4/15
8583/8583 [==============================] - 5s 606us/step - loss: 0.2684 - acc: 0.8891
Epoch 5/15
8583/8583 [==============================] - 5s 603us/step - loss: 0.2516 - acc: 0.8974
Epoch 6/15
8583/8583 [==============================] - 5s 609us/step - loss: 0.2377 - acc: 0.8996
Epoch 7/15
8583/8583 [==============================] - 5s 614us/step - loss: 0.2260 - acc: 0.9060
Epoch 8/15
8583/8583 [==============================] - 5s 611us/step - loss: 0.2157 - acc: 0.9113
Epoch 9/15
8583/8583 [==============================] - 5s 600us/step - loss: 0.2033 - acc: 0.9179
Epoch 10/15
8583/8583 [==============================] - 5s 600us/step - loss: 0.1917 - acc: 0.9211
Epoch 11/

Extracting a validation set, and measuring score and accuracy.

In [10]:
Y_pred = model.predict_classes(X_test,batch_size = batch_size)

In [11]:
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

confusion matrix [[1571  142]
 [ 215  218]]
              precision    recall  f1-score   support

           0       0.88      0.92      0.90      1713
           1       0.61      0.50      0.55       433

   micro avg       0.83      0.83      0.83      2146
   macro avg       0.74      0.71      0.72      2146
weighted avg       0.82      0.83      0.83      2146



Finally measuring the number of correct guesses.  It is clear that finding negative tweets (**class 0**) goes very well (**recall 0.92**) for the Network but deciding whether is positive (**class 1**) is not really (**recall 0.52**). My educated guess here is that the positive training set is dramatically smaller than the negative, hence the "bad" results for positive tweets.

As expected accuracy for positive data is vary low compare to negative, Lets try to solve this problem.

**1. Up-sample Minority Class**

Up-sampling is the process of randomly duplicating observations from the minority class in order to reinforce its signal.
There are several heuristics for doing so, but the most common way is to simply resample with replacement.

In [12]:
# Separate majority and minority classes
data_majority = data[data['sentiment'] == 'Negative']
data_minority = data[data['sentiment'] == 'Positive']

bias = data_minority.shape[0]/data_majority.shape[0]
# lets split train/test data first then 
train = pd.concat([data_majority.sample(frac=0.8,random_state=200),
         data_minority.sample(frac=0.8,random_state=200)])
test = pd.concat([data_majority.drop(data_majority.sample(frac=0.8,random_state=200).index),
        data_minority.drop(data_minority.sample(frac=0.8,random_state=200).index)])

train = shuffle(train)
test = shuffle(test)

In [13]:
print('positive data in training:',(train.sentiment == 'Positive').sum())
print('negative data in training:',(train.sentiment == 'Negative').sum())
print('positive data in test:',(test.sentiment == 'Positive').sum())
print('negative data in test:',(test.sentiment == 'Negative').sum())


positive data in training: 1789
negative data in training: 6794
positive data in test: 447
negative data in test: 1699


In [14]:
# Separate majority and minority classes in training data for upsampling 
data_majority = train[train['sentiment'] == 'Negative']
data_minority = train[train['sentiment'] == 'Positive']

print("majority class before upsample:",data_majority.shape)
print("minority class before upsample:",data_minority.shape)

# Upsample minority class
data_minority_upsampled = resample(data_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples= data_majority.shape[0],    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
data_upsampled = pd.concat([data_majority, data_minority_upsampled])
 
# Display new class counts
print("After upsampling\n",data_upsampled.sentiment.value_counts(),sep = "")

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values) # training with whole data

X_train = tokenizer.texts_to_sequences(data_upsampled['text'].values)
X_train = pad_sequences(X_train,maxlen=29)
Y_train = pd.get_dummies(data_upsampled['sentiment']).values
print('x_train shape:',X_train.shape)

X_test = tokenizer.texts_to_sequences(test['text'].values)
X_test = pad_sequences(X_test,maxlen=29)
Y_test = pd.get_dummies(test['sentiment']).values
print("x_test shape", X_test.shape)

majority class before upsample: (6794, 2)
minority class before upsample: (1789, 2)
After upsampling
Positive    6794
Negative    6794
Name: sentiment, dtype: int64
x_train shape: (13588, 29)
x_test shape (2146, 29)


In [15]:
# model
embed_dim = 128
lstm_out = 192

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X_train.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.4, recurrent_dropout=0.4))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 29, 128)           256000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 29, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 192)               246528    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 386       
Total params: 502,914
Trainable params: 502,914
Non-trainable params: 0
_________________________________________________________________
None


Here we train the Network. We should run much more than 15 epoch, but I would have to wait forever for kaggle, so it is 15 for now.

In [16]:
batch_size = 128
# also adding weights
class_weights = {0: 1 ,
                1: 1.6/bias }
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 1,
          class_weight=class_weights)

Epoch 1/15
13588/13588 [==============================] - 10s 700us/step - loss: 1.2697 - acc: 0.5703
Epoch 2/15
13588/13588 [==============================] - 9s 627us/step - loss: 0.7914 - acc: 0.7612
Epoch 3/15
13588/13588 [==============================] - 8s 621us/step - loss: 0.6597 - acc: 0.8159
Epoch 4/15
13588/13588 [==============================] - 8s 623us/step - loss: 0.5813 - acc: 0.8403
Epoch 5/15
13588/13588 [==============================] - 8s 621us/step - loss: 0.5450 - acc: 0.8534
Epoch 6/15
13588/13588 [==============================] - 8s 622us/step - loss: 0.4764 - acc: 0.8728
Epoch 7/15
13588/13588 [==============================] - 8s 620us/step - loss: 0.4493 - acc: 0.8817
Epoch 8/15
13588/13588 [==============================] - 8s 624us/step - loss: 0.4243 - acc: 0.8903
Epoch 9/15
13588/13588 [==============================] - 8s 624us/step - loss: 0.3913 - acc: 0.8970
Epoch 10/15
13588/13588 [==============================] - 8s 625us/step - loss: 0.3829 - 

In [17]:
Y_pred = model.predict_classes(X_test,batch_size = batch_size)
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

confusion matrix [[1379  320]
 [ 125  322]]
              precision    recall  f1-score   support

           0       0.92      0.81      0.86      1699
           1       0.50      0.72      0.59       447

   micro avg       0.79      0.79      0.79      2146
   macro avg       0.71      0.77      0.73      2146
weighted avg       0.83      0.79      0.80      2146



So the class imbalance is reduced significantly recall value for positive tweets (Class 1) improved from 0.54 to 0.77. It is alwayes not possible to reduce it compleatly. 

You may also noticed that the recall value for Negative tweets also decreased from 0.90 to 0.78  but this can be improved using training model to more epocs and tuning the hyperparameters.

In [18]:
# running model to few more epochs
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 1,
          class_weight=class_weights)
Y_pred = model.predict_classes(X_test,batch_size = batch_size)
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

Epoch 1/15
13588/13588 [==============================] - 8s 621us/step - loss: 0.3102 - acc: 0.9200
Epoch 2/15
13588/13588 [==============================] - 8s 621us/step - loss: 0.2973 - acc: 0.9262
Epoch 3/15
13588/13588 [==============================] - 8s 622us/step - loss: 0.2899 - acc: 0.9290
Epoch 4/15
13588/13588 [==============================] - 8s 622us/step - loss: 0.2856 - acc: 0.9268
Epoch 5/15
13588/13588 [==============================] - 8s 622us/step - loss: 0.2863 - acc: 0.9297
Epoch 6/15
13588/13588 [==============================] - 8s 623us/step - loss: 0.2798 - acc: 0.9299
Epoch 7/15
13588/13588 [==============================] - 8s 621us/step - loss: 0.2769 - acc: 0.9310
Epoch 8/15
13588/13588 [==============================] - 8s 623us/step - loss: 0.2695 - acc: 0.9339
Epoch 9/15
13588/13588 [==============================] - 8s 619us/step - loss: 0.2647 - acc: 0.9340
Epoch 10/15
13588/13588 [==============================] - 8s 620us/step - loss: 0.2619 - a

In [19]:
twt = ['keep up the good work']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=29, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0 381  47   1 137 464]]
positive


1. ## Tuning the hyper-parameters using gridsearch

In [20]:
# from sklearn.model_selection import GridSearchCV
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.wrappers.scikit_learn import KerasClassifier

In [21]:
# # Function to create model, required for KerasClassifier
# def create_model(dropout_rate = 0.0):
#     # create model
#     embed_dim = 128
#     lstm_out = 192
#     model = Sequential()
#     model.add(Embedding(max_fatures, embed_dim,input_length = X_train.shape[1]))
#     model.add(SpatialDropout1D(dropout_rate))
#     model.add(LSTM(lstm_out, dropout=dropout_rate, recurrent_dropout=dropout_rate))
#     model.add(Dense(2,activation='softmax'))
#     model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
# #     print(model.summary())
#     return model
# # fix random seed for reproducibility
# seed = 7
# np.random.seed(seed)

# model = KerasClassifier(build_fn=create_model, verbose=2,epochs=30, batch_size=128)
# # define the grid search parameters
# # batch_size = [128]
# # epochs = [5]
# dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# # class_weight = [{0: 1, 1: 1/bias},{0: 1, 1: 1.2/bias},{0: 1, 1: 1.5/bias},{0: 1, 1: 1.8/bias}]
# param_grid = dict(dropout_rate = dropout_rate)
# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
# grid_result = grid.fit(X_train, Y_train)
# # summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

Finally measuring the number of correct guesses.  It is clear that finding negative tweets goes very well for the Network but deciding whether is positive is not really. My educated guess here is that the positive training set is dramatically smaller than the negative, hence the "bad" results for positive tweets.

its 10% increase in positive accuracy from 56% to 66%

In [22]:
twt = ['inaccurate facts, dont vote for him']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=29, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0 1288   43  200   14
    74]]
positive
